import pandas as pd
import numpy as np
import math

In [238]:
df = pd.read_csv("salaries.csv")
df

,company,job,degree,salary_more_then_100k
0,google,sales executive,bachelors,0
1,google,sales executive,masters,0
2,google,business manager,bachelors,1
3,google,business manager,masters,1
4,google,computer programmer,bachelors,0
5,google,computer programmer,masters,1
6,abc pharma,sales executive,masters,0
7,abc pharma,computer programmer,bachelors,0
8,abc pharma,business manager,bachelors,0
9,abc pharma,business manager,masters,1


In [239]:
from sklearn.preprocessing import LabelEncoder

enc1 = LabelEncoder()
company_enc = enc1.fit_transform(df['company'])
job_enc = enc1.fit_transform(df['job'])
degree_enc = enc1.fit_transform(df['degree'])
df['company_enc'] = company_enc
df['job_enc'] = job_enc
df['degree_enc'] = degree_enc
X = df[['company_enc', 'job_enc', 'degree_enc']]
y = df['salary_more_then_100k']
df = df.drop(['company', 'job', 'degree'], axis = 1)
df = df[['company_enc', 'job_enc', 'degree_enc', 'salary_more_then_100k']]

In [240]:
df

,company_enc,job_enc,degree_enc,salary_more_then_100k
0,2,2,0,0
1,2,2,1,0
2,2,0,0,1
3,2,0,1,1
4,2,1,0,0
5,2,1,1,1
6,0,2,1,0
7,0,1,0,0
8,0,0,0,0
9,0,0,1,1


In [241]:
class TreeNode:
    col = -1
    isEnd = False
    yes = "no"
    def __init__(self, col_val):
        self.col_val = col_val
        self.node_arr = []

In [242]:
def min_gini(df):
    X = df.iloc[:, 0:len(df.columns) - 1]
    y = df['salary_more_then_100k']  
    gini_scores = []  
    for col in X.columns:
        col_values = X[col].values
        unique_vals, counts = np.unique(col_values, return_counts=True)
        pos_counts = np.array([np.sum(y[col_values == val] == 1) for val in unique_vals])
        total_counts = counts
        pos_prob = pos_counts / total_counts
        neg_prob = 1 - pos_prob
        gini_per_val = 1 - (pos_prob**2 + neg_prob**2)
        gini = np.sum((total_counts / len(col_values)) * gini_per_val)
        gini_scores.append(gini)
    max_gini_idx = np.argmin(gini_scores)
    return max_gini_idx, gini_scores[max_gini_idx]

In [243]:
def split_df(df, node, colm):
    new_dfs = []
    for val in df.iloc[:, colm].unique():
        new_node = TreeNode(val)
        t_df = df[df.iloc[:, colm] == val]
        t_df = t_df.reset_index(drop = True)
        node.node_arr.append(new_node)
        new_dfs.append(t_df.drop(t_df.columns[colm], axis = 1))
    return new_dfs

In [263]:
def training(df, node):
    maxm, maxi = min_gini(df)
    if (maxi == 0):
        node.isEnd = True
        if (df.iloc[0, -1] == 1):
            node.yes = "yes"
        return
    new_dfs = split_df(df, node, maxm)
    node.col = df.columns[maxm]
    for i in range(0, len(new_dfs)):
        training(new_dfs[i], node.node_arr[i])
    return

In [249]:
root = TreeNode(-1)
training(df, root)

    company_enc  job_enc  degree_enc  salary_more_then_100k
0             2        2           0                      0
1             2        2           1                      0
2             2        0           0                      1
3             2        0           1                      1
4             2        1           0                      0
5             2        1           1                      1
6             0        2           1                      0
7             0        1           0                      0
8             0        0           0                      0
9             0        0           1                      1
10            1        2           0                      1
11            1        2           1                      1
12            1        0           0                      1
13            1        0           1                      1
14            1        1           0                      1
15            1        1           1    

In [261]:
def traverse(root, vec):
    if (root.isEnd):
        print(root.yes)
        return
    print(root.col, root.col_val)
    for node in root.node_arr:
        if (node.col_val == vec[root.col]):
            traverse(node)
    return

In [262]:
traverse(root)

company_enc -1
job_enc 2
no
yes
no
job_enc 0
no
no
no
yes
